In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Step 3: Quick EDA (Exploratory Data Analysis)
print(dataset.info())
print(dataset.describe())
sns.pairplot(dataset)
plt.show()

# Check categorical variables
print(dataset['sex'].value_counts())
print(dataset['smoker'].value_counts())
print(dataset['region'].value_counts())


In [ ]:
# Step 4: Encode categorical variables
dataset_encoded = dataset.copy()
dataset_encoded['sex'] = dataset_encoded['sex'].map({'male':0, 'female':1})
dataset_encoded['smoker'] = dataset_encoded['smoker'].map({'no':0, 'yes':1})
dataset_encoded = pd.get_dummies(dataset_encoded, columns=['region'], drop_first=True)

dataset_encoded.head()


In [ ]:
# Step 5: Split dataset
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(dataset_encoded, test_size=0.2, random_state=42)

# Step 6: Separate labels
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')


In [ ]:
# Step 7: Normalize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_dataset_scaled = scaler.fit_transform(train_dataset)
test_dataset_scaled = scaler.transform(test_dataset)


In [ ]:
# Step 8: Build the model
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[train_dataset_scaled.shape[1]]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', 'mse']
    )
    return model

model = build_model()


In [ ]:
# Step 9: Train the model
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    train_dataset_scaled, train_labels,
    validation_split=0.2,
    epochs=500,
    batch_size=32,
    verbose=0,
    callbacks=[early_stop]
)

# Plot training history
plt.figure(figsize=(8,6))
plt.xlabel('Epoch')
plt.ylabel('Mean Abs Error [expenses]')
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.legend()
plt.show()


In [ ]:
# Step 10: Evaluate the model on test set
loss, mae, mse = model.evaluate(test_dataset_scaled, test_labels, verbose=2)
print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

# Step 11: Predict and plot results
test_predictions = model.predict(test_dataset_scaled).flatten()
a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
plt.plot(lims, lims)
plt.show()
